In [1]:
import pandas as pd
from Acesso import Login
from Query import Query
from glob import glob
from datetime import datetime,timedelta

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)

In [78]:
querys={
    
    'Produto':
    
    """
    
    SELECT * FROM netfeira.vw_produto
    
    """,
    
    'Cliente':
    
    """
    
    SELECT * FROM netfeira.vw_cliente
    
    """,
    
    'Segmento':
    
    """
    
    SELECT * FROM netfeira.vw_segmento
    
    """,
    
    'Margens':
    
    """
    
    SELECT * FROM netfeira.vw_margens
    
    """,
    
    'Vendedor':
    
    """
    
    SELECT * FROM netfeira.vw_vendedor
    
    """,
    
    'Supervisor':
    
    """
    
    SELECT * FROM netfeira.vw_supervisor
    
    """
}

In [94]:
tabelas_df=sql.CriarTabela(kwargs=querys)

In [95]:
mes=datetime.now().month-1

mes=mes if mes>0 else 12

ano=datetime.now().year if mes>0 else datetime.now().year-1

In [100]:
tabelas_df['Consolidado']=tabelas_df['Margens'].loc[(tabelas_df['Margens']['ID Situação']=='FA')&(tabelas_df['Margens']['Data de Faturamento'].dt.year==ano)&(tabelas_df['Margens']['Data de Faturamento'].dt.month==mes)&(tabelas_df['Margens']['Tipo de Operação']=='VENDAS')]

In [133]:
tabelas_df['Analise']=tabelas_df['Consolidado'][['SKU','Total Venda','Margem CUE','MG CUE','Custo CUE']].groupby(['SKU'],as_index=False).sum()

tabelas_df['Analise']['MG Bruta']=round(tabelas_df['Analise']['Margem CUE']/tabelas_df['Analise']['Total Venda'],4)*100

tabelas_df['Analise']['MG CTB CUE']=round(tabelas_df['Analise']['MG CUE']/tabelas_df['Analise']['Total Venda'],4)*100

tabelas_df['Analise'].sort_values('MG CTB CUE',ascending=True,inplace=True)

codigos=tabelas_df['Analise']['SKU'].loc[tabelas_df['Analise']['MG CTB CUE']<18].unique().tolist()

tabelas_df['Analise'].loc[tabelas_df['Analise']['MG CTB CUE']<18]

,SKU,Total Venda,Margem CUE,MG CUE,Custo CUE,MG Bruta,MG CTB CUE
178,1113,70.17,-465.43,-423.97,535.60,-663.29,-604.20
223,9344,5.00,-24.05,-17.88,29.05,-481.00,-357.60
199,1159,4.00,-11.66,-11.66,15.66,-291.50,-291.50
82,665,108.00,-260.00,-260.00,368.00,-240.74,-240.74
17,218,2265.30,-5325.89,-5325.89,7591.19,-235.11,-235.11
187,1130,97.90,-230.51,-200.13,328.41,-235.45,-204.42
309,9566,2564.10,-6294.09,-5231.07,8858.19,-245.47,-204.01
170,1097,60.80,-140.60,-116.43,201.40,-231.25,-191.50
172,1100,25.40,-51.60,-48.52,77.00,-203.15,-191.02
171,1098,18.60,-34.40,-28.04,53.00,-184.95,-150.75


In [135]:
tabelas_df['Tabelas']=tabelas_df['Consolidado'][['Tabelas','Total Venda','Margem CUE','MG CUE','Custo CUE']].loc[tabelas_df['Consolidado']['SKU'].isin(codigos)].groupby('Tabelas',as_index=False).sum()

tabelas_df['Tabelas']['MG Bruta']=round(tabelas_df['Tabelas']['Margem CUE']/tabelas_df['Tabelas']['Total Venda'],4)*100

tabelas_df['Tabelas']['MG CTB CUE']=round(tabelas_df['Tabelas']['MG CUE']/tabelas_df['Tabelas']['Total Venda'],4)*100

tabelas_df['Tabelas'].sort_values('MG CTB CUE',ascending=True,inplace=True,ignore_index=True)

tabelas=tabelas_df['Tabelas']['Tabelas'].loc[tabelas_df['Tabelas']['MG CTB CUE']<18].unique().tolist()

tabelas_df['Tabelas'].loc[tabelas_df['Tabelas']['MG CTB CUE']<18]

,Tabelas,Total Venda,Margem CUE,MG CUE,Custo CUE,MG Bruta,MG CTB CUE
0,FEFO,11906.85,-29585.00,-27055.50,41491.85,-248.47,-227.23
1,YAMAUCHI,455.40,-452.58,-475.32,907.98,-99.38,-104.37
2,LOPES,2103.66,-1830.92,-1936.09,3934.58,-87.03,-92.03
3,SONDA,21030.98,-3770.76,-4842.74,24801.74,-17.93,-23.03
4,PEGE PESE,14134.00,-2250.41,-2957.09,16384.41,-15.92,-20.92
5,TABELA DE NEGOCIAÇÃO,137858.87,11838.37,3941.79,126020.50,8.59,2.86
6,TABELA OBA,11206.15,1879.85,1145.91,9326.30,16.78,10.23
7,TABELA FUNCIONÁRIO,1520.21,105.09,206.35,1415.12,6.91,13.57
8,PASTORINHO,13051.38,3957.41,1978.65,9093.97,30.32,15.16
9,TABELA DE PREÇO,619320.64,185570.70,109725.12,433749.94,29.96,17.72


In [136]:
tabelas_df['Lista']=tabelas_df['Consolidado'][['ID Cliente','Total Venda','Margem CUE','MG CUE','Custo CUE']].loc[(tabelas_df['Consolidado']['SKU'].isin(codigos))&tabelas_df['Consolidado']['Tabelas'].isin(tabelas)].groupby(['ID Cliente'],as_index=False).sum()

tabelas_df['Lista']['MG Bruta']=round(tabelas_df['Lista']['Margem CUE']/tabelas_df['Lista']['Total Venda'],4)*100

tabelas_df['Lista']['MG CTB CUE']=round(tabelas_df['Lista']['MG CUE']/tabelas_df['Lista']['Total Venda'],4)*100

tabelas_df['Lista'].sort_values('MG CTB CUE',ascending=True,inplace=True,ignore_index=True)

clientes=tabelas_df['Lista']['ID Cliente'].loc[tabelas_df['Lista']['MG CTB CUE']<18].unique().tolist()

tabelas_df['Lista'].loc[tabelas_df['Lista']['MG CTB CUE']<18]

,ID Cliente,Total Venda,Margem CUE,MG CUE,Custo CUE,MG Bruta,MG CTB CUE
0,22409,7.00,-65.10,-59.32,72.10,-930.00,-847.43
1,22380,7.00,-65.10,-59.32,72.10,-930.00,-847.43
2,28250,6.00,-55.80,-50.84,61.80,-930.00,-847.33
3,22082,11.00,-71.40,-64.79,82.40,-649.09,-589.00
4,23382,7.00,-35.42,-33.16,42.42,-506.00,-473.71
...,...,...,...,...,...,...,...
463,27785,331.50,83.40,58.51,248.10,25.16,17.65
464,26678,526.08,142.50,93.20,383.58,27.09,17.72
465,22459,1872.96,531.60,334.35,1341.36,28.38,17.85
466,6249,9501.12,3246.87,1701.78,6254.25,34.17,17.91


In [137]:
tabelas_df['Historico']=tabelas_df['Consolidado'][['ID Cliente','Pedido','Tabelas','SKU','Total Venda','Margem CUE','MG CUE','Custo CUE']].loc[(tabelas_df['Consolidado']['SKU'].isin(codigos))&tabelas_df['Consolidado']['Tabelas'].isin(tabelas)&(tabelas_df['Consolidado']['ID Cliente'].isin(clientes))]

tabelas_df['Historico']['MG Bruta']=round(tabelas_df['Historico']['Margem CUE']/tabelas_df['Historico']['Total Venda'],4)*100

tabelas_df['Historico']['MG CTB CUE']=round(tabelas_df['Historico']['MG CUE']/tabelas_df['Historico']['Total Venda'],4)*100

tabelas_df['Historico'].sort_values('MG CTB CUE',ascending=True,inplace=True,ignore_index=True)

tabelas_df['Historico']=tabelas_df['Historico'].loc[tabelas_df['Historico']['MG CTB CUE']<18]

tabelas_df['Historico'].loc[~tabelas_df['Historico']['Tabelas'].str.contains('PREÇO')]

,ID Cliente,Pedido,Tabelas,SKU,Total Venda,Margem CUE,MG CUE,Custo CUE,MG Bruta,MG CTB CUE
0,20047,435162,FEFO,348,1.0,-24.0,-24.00,25.0,-2400.00,-2400.00
1,20047,435162,FEFO,347,1.0,-18.0,-18.00,19.0,-1800.00,-1800.00
2,22380,433142,FEFO,1113,7.0,-65.1,-59.32,72.1,-930.00,-847.43
3,22082,433162,FEFO,1113,7.0,-65.1,-59.32,72.1,-930.00,-847.43
4,22409,433146,FEFO,1113,7.0,-65.1,-59.32,72.1,-930.00,-847.43
...,...,...,...,...,...,...,...,...,...,...
1122,16939,435849,PEGE PESE,350,1179.0,261.0,202.05,918.0,22.14,17.14
1123,4630,435848,PEGE PESE,350,589.5,130.5,101.02,459.0,22.14,17.14
1131,27785,431545,TABELA DE NEGOCIAÇÃO,9434,110.5,27.8,19.32,82.7,25.16,17.48
1132,27785,431545,TABELA DE NEGOCIAÇÃO,9435,110.5,27.8,19.32,82.7,25.16,17.48


In [146]:
tabelas_df['Temp']=tabelas_df['Historico'].loc[tabelas_df['Historico']['Tabelas'].str.contains('PREÇO')]

tabelas_df['Temp']=tabelas_df['Temp'][['SKU','Total Venda','Margem CUE','MG CUE','Custo CUE']].groupby(['SKU'],as_index=False).sum()

tabelas_df['Temp']['MG Bruta']=round(tabelas_df['Temp']['Margem CUE']/tabelas_df['Temp']['Total Venda'],4)*100

tabelas_df['Temp']['MG CTB CUE']=round(tabelas_df['Temp']['MG CUE']/tabelas_df['Temp']['Total Venda'],4)*100

tabelas_df['Temp'].sort_values('MG CTB CUE',ascending=True,inplace=True,ignore_index=True)

tabelas_df['Temp']=tabelas_df['Temp'].merge(tabelas_df['Produto'][['SKU','Produto','Fabricante']],on='SKU',how='inner')[['SKU','Produto','Fabricante','Total Venda','Margem CUE','MG CUE','MG Bruta','MG CTB CUE','Custo CUE']]

tabelas_df['Temp'].loc[tabelas_df['Temp']['Fabricante']!='MCCAIN']

,SKU,Produto,Fabricante,Total Venda,Margem CUE,MG CUE,MG Bruta,MG CTB CUE,Custo CUE
0,9591,CONGELADO EDAMAME 300G DE MARCHI,FRAGOLE,477.83,-196.67,-177.25,-41.16,-37.09,674.50
1,660,"PALMITO INTEIRO (1,8 KG)",CONTE,2600.87,839.00,197.53,32.26,7.59,1761.87
2,646,"PALMITO FATIADO (1,8 KG)",CONTE,136.39,51.43,13.48,37.71,9.88,84.96
5,645,"PALMITO PICADO (1,8 KG)",CONTE,372.15,136.58,48.07,36.70,12.92,235.57
7,1076,CONCENTRADO MANGA PARA REFRESCO 5 LT,DE MARCHI,1692.24,579.04,219.08,34.22,12.95,1113.20
8,1062,CONCENTRADO CAJU PARA REFRESCO 5 LT,DE MARCHI,1873.75,641.25,242.58,34.22,12.95,1232.50
9,1068,CONCENTRADO GOIABA PARA REFRESCO 5 LT,DE MARCHI,1423.04,493.04,185.07,34.65,13.01,930.00
20,9366,"ACAI BUFALO CONGELADO 3,3 KG",DE MARCHI,680.35,240.69,101.18,35.38,14.87,439.66
22,9568,SUCO PINK LEMONADE 300ML DEMARCHI,DE MARCHI,163.86,43.22,25.30,26.38,15.44,120.64
23,730,CONGELADO MANDIOCA PALITO DM 1.2 KG,DE MARCHI,511.10,134.75,79.47,26.36,15.55,376.35
